## Preprocessing Sction.
This section is inspired by the preprocessing of another computer scientist who was using the same dataset to do their own experiments on it.

Link to their repositry: [TanushGoel / Breast-Histopathology-IDC-Classification](https://github.com/TanushGoel/Breast-Histopathology-IDC-Classification)

In [1]:
# Testing that the machine recognizes the GPU
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import seaborn as sns
import os
from os.path import isfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import utils
import itertools
import shutil
np.random.seed(42)
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
print(tf.__version__)
print(tf.keras.__version__)

2.9.1
2.9.0


In [8]:
current_dir = os.getcwd()

In [ ]:
#create new all image dir folder 
all_images_dir = 'all_images_dir_AUG'
os.chdir('./breast-histopathology-images/')
if(os.path.exists(f'./breast-histopathology-images/{all_images_dir}')):
    os.mkdir(all_images_dir)

    # Create Positive subdirectory within all images
    os.mkdir('all_images_dir_AUG/1')
    os.mkdir('all_images_dir_AUG/0')

In [5]:

# to get rid of changing the directory more than once
# os.chdir(current_dir)

###### Mostly, taken from the other author's notebook. ######

# all_images_dir = 'all_images_dir_AUG'
# os.chdir('./breast-histopathology-images/')
# copies all images from their seperate folders into the same 
# folder called all_images_dir


# create a list of all patient id's
# each patient id folder has 2 sub folders --> folder 0 and folder 1

# Example:
    # '10285'
        # '0'
        # '1'
        
all_pos_len = len(os.listdir(f'{all_images_dir}/1'))
all_neg_len = len(os.listdir(f'{all_images_dir}/0'))


if all_pos_len == 0 or all_neg_len == 0:
    cur_dir = os.getcwd()
    patient_list = os.listdir(cur_dir)

    for patient in patient_list:

      try:

        path_0 = str(patient) + '/0'
        path_1 = str(patient) + '/1'

        # create a list of all files in folder 0
        file_list_0 = os.listdir(path_0)
        # create a list of list all file in folder 1
        file_list_1 = os.listdir(path_1)

        # move the 0 images to all_images_dir
        for fname in file_list_0:

            # source path to image
            src = os.path.join(path_0, fname)
            # destination path to image
            dst = os.path.join(all_images_dir, '0', fname)
            # copy the image from the source to the destination
            shutil.copyfile(src, dst)

        # move the 1 images to all_images_dir
        for fname in file_list_1:

            # source path to image
            src = os.path.join(path_1, fname)
            # destination path to image
            dst = os.path.join(all_images_dir, '1', fname)
            # copy the image from the source to the destination
            shutil.copyfile(src, dst)

      except:
         print(patient, 'cannot be copied into proper folder')
else:
    print("Data already exists. Skipped copying")

FileNotFoundError: [Errno 2] No such file or directory: 'all_images_dir/1'

In [ ]:
# Check how many images are in each folder, to make sure we have correct data
print('Total:', all_pos_len + all_neg_len)
print('Positive:', all_pos_len)
print('Negative:', all_neg_len)

In [ ]:
###### Mostly, taken from the other author's notebook. ######

# Create a dataframe containing all the information

image_list_1 = os.listdir(f'{all_images_dir}/1')
df_data_1 = pd.DataFrame(image_list_1, columns=['image_id'])

image_list_0 = os.listdir(f'{all_images_dir}/0')
df_data_0 = pd.DataFrame(image_list_0, columns=['image_id'])

df_data = pd.concat([df_data_1, df_data_0])

In [ ]:
# Validating that the data now exists in the dataframe
df_data.head()

In [ ]:
###### Mostly, taken from the other author's notebook. ######

# Define Helper Functions

# Each file name has this format:
# '14211_idx5_x2401_y1301_class1.png'

def extract_patient_id(x):
    # split into a list
    a = x.split('_')
    # the id is the first index in the list
    patient_id = a[0]
    
    return patient_id

def extract_target(x):
    # split into a list
    a = x.split('_')
    # the target is part of the string in index 4
    b = a[4]
    # the ytarget i.e. 1 or 2 is the 5th index of the string --> class1
    target = b[5]
    
    return target

# extract the patient id

# create a new column called 'patient_id'
df_data['patient_id'] = df_data['image_id'].apply(extract_patient_id)
# create a new column called 'target'
df_data['target'] = df_data['image_id'].apply(extract_target)

df_data.head()

### The class distribution of the dataset.
There are more negative cases than the positive ones. This makes the dataset imbalanced since the nature of the problem doesn't require that the number of negative cases to be more or less than the positive ones.

In [ ]:
# Showing the class distribution of the dataset, We find that the negative is way more than the positives.
sns.countplot(x=df_data.target).set_title("Class Distributions")

# Train / Test / Validation

In [ ]:
from sklearn.model_selection import train_test_split

y = df_data['target']

_, df_test = train_test_split(df_data, test_size=0.03603, random_state=42, stratify=y)

_, df_val = train_test_split(df_data, test_size=0.0036025, random_state=42, stratify=y)

print('Valid:', df_val.shape[0])
print('Test:', df_test.shape[0])

In [ ]:
sns.countplot(x=df_val['target']).set_title('Valid Class Distributions')

In [ ]:
sns.countplot(x=df_test['target']).set_title('Test Class Distributions')

In [ ]:
# Improved the speed of the identification through using a dictionary... Dramatically.
val_dict = df_val.set_index('image_id').T.to_dict('list')
test_dict = df_test.set_index('image_id').T.to_dict('list')

def identify_train_val_and_test_rows(x):
    if str(x) in val_dict:
        return 'val'
    elif str(x) in test_dict:
        return 'test'
    else:
        return 'train'

# create a new colum that is a copy of the image_id column
df_data['train_or_val_or_test'] = df_data['image_id']
# apply the function to this new column
df_data['train_or_val_or_test'] = df_data['train_or_val_or_test'].apply(identify_train_val_and_test_rows)
   
# filter out train rows
df_train = df_data[df_data['train_or_val_or_test'] == 'train']

print('Train:', len(df_train))
print('Valid:', len(df_val))
print('Test:', len(df_test))

In [ ]:
sns.countplot(x=df_train['target']).set_title('Train Class Distributions')

In [ ]:
# Set the image_id as the index in df_data
df_data.set_index('image_id', inplace=True)
df_data.head()

In [ ]:
# Make Train, Valid, and Test Folders


# Create folders within the train, valid, and test folders
# Inside each folder we create seperate folders for each class

# train_dir
train_dir = os.path.join('train_dir_AUG')
if not os.path.isdir(train_dir):
    os.mkdir(train_dir)
    # create new folders inside train_dir
    os.mkdir(os.path.join(train_dir, '1'))
    os.mkdir(os.path.join(train_dir, '0'))
    
# val_dir
val_dir = os.path.join('val_dir_AUC')
if not os.path.isdir(val_dir):
    os.mkdir(val_dir)
    # Create new folders inside val_dir
    os.mkdir(os.path.join(val_dir, '1'))
    os.mkdir(os.path.join(val_dir, '0'))

# test_dir
test_dir = os.path.join('test_dir_AUG')
if not os.path.isdir(test_dir):
    os.mkdir(test_dir)
    # Create new folders inside test_dir
    os.mkdir(os.path.join(test_dir, '1'))
    os.mkdir(os.path.join(test_dir, '0'))

In [ ]:
# Transfering the data from the starting directory to the training directory
if len(os.listdir(train_dir)) == 0:
    # Get a dict of train, val, and test images
    train_dict = df_train.set_index('image_id').T.to_dict('list')
    test_dict = df_test.set_index('image_id').T.to_dict('list')
    val_dict = df_val.set_index('image_id').T.to_dict('list')

    # Transfer the train images
    for image in train_dict:
        pat_id = df_data.loc[image,'patient_id']
        label = df_data.loc[image,'target']
        src = pat_id + '/' + label + '/' + image

        # destination path to image
        dst = os.path.join(train_dir, label, image)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)


    # Transfer the val images
    for image in val_dict:
        pat_id = df_data.loc[image,'patient_id']
        label = df_data.loc[image,'target']
        src = pat_id + '/' + label + '/' + image

        # destination path to image
        dst = os.path.join(val_dir, label, image)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)


    # Transfer the test images
    for image in test_dict:
        pat_id = df_data.loc[image,'patient_id']
        label = df_data.loc[image,'target']
        src = pat_id + '/' + label + '/' + image

        # destination path to image
        dst = os.path.join(test_dir, label, image)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
else:
    print("Data already exists (Hopefully), it might not be transfered correctly.")

In [ ]:
print('Train Total:', len(os.listdir('train_dir/1'))+len(os.listdir('train_dir/0')))
print('Valid Total:', len(os.listdir('val_dir/1'))+len(os.listdir('val_dir/0')))
print('Test Total:', len(os.listdir('test_dir/1'))+len(os.listdir('test_dir/0')), '\n')

print('Train Negative:', len(os.listdir('train_dir/0')))
print('Train Positive:', len(os.listdir('train_dir/1')), '\n')

print('Valid Negative:', len(os.listdir('val_dir/0')))
print('Valid Positive:', len(os.listdir('val_dir/1')), '\n')

print('Test Negative:', len(os.listdir('test_dir/0')))
print('Test Positive:', len(os.listdir('test_dir/1')))

# Agumentation


In [ ]:
image_size = 100

from keras.preprocessing.image import ImageDataGenerator

# create temporary directories here and delete these directories later
aug_dir = 'aug_dir'
os.mkdir(aug_dir)
# create a dir within the base dir to store images of the same class
img_dir = os.path.join(aug_dir, 'img_dir')
os.mkdir(img_dir)

# list all images in that directory
img_list = os.listdir('train_dir/1')

# copy images from the class all images directory to the image directory
for fname in img_list:
    # source path to image
    src = os.path.join('train_dir/1', fname)
    # destination path to image
    dst = os.path.join(img_dir, fname)
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)


# point to a dir containing the images and not to the images themselves
path = 'aug_dir'
save_path = 'train_dir/1'

# create a data generator
datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.125,
    height_shift_range=0.125,
    zoom_range=[1.2, 1.325],
    fill_mode='reflect')

batch_size = 25

aug_datagen = datagen.flow_from_directory(path,
                                          save_to_dir=save_path,
                                          save_format='jpg',
                                          target_size=(image_size, image_size),
                                          batch_size=batch_size)

# generate the augmented images and add them to the folder of all images

num_aug_images_wanted = len(os.listdir('train_dir/0')) # total number of images wanted in each class

num_files = len(os.listdir(img_dir))
num_batches = int(np.ceil((num_aug_images_wanted-num_files)/batch_size))

for i in range(0, num_batches):
  next(aug_datagen)

# delete temporary directory with the raw image files
shutil.rmtree('aug_dir')


# Check how many train images we have in each folder

print('Train Total:', len(os.listdir('train_dir/1'))+len(os.listdir('train_dir/0')))
print('Train Positive:', len(os.listdir('train_dir/1')))
print('Train Negative:', len(os.listdir('train_dir/0')))

### Create train / test / validation batches

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_path = 'train_dir_AUG'
valid_path = 'val_dir_AUG'
test_path = 'test_dir_AUG'

num_train_samples = len(df_train)
num_val_samples = len(df_val)

train_batch_size = 500
val_batch_size = 500

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=keras.applications.inception_v3.preprocess_input, 
                             horizontal_flip=True,
                             vertical_flip=True)
image_size=100

train_batches_bench = datagen.flow_from_directory(train_path, 
                                                  target_size=(image_size, image_size),
                                                  batch_size=train_batch_size)

valid_batches_bench = datagen.flow_from_directory(valid_path,  
                                                  target_size=(image_size, image_size),
                                                  batch_size=val_batch_size)

test_batches_bench = datagen.flow_from_directory(test_path,  
                                                 target_size=(image_size, image_size),
                                                 batch_size=1,
                                                 shuffle=False) # test dataset should not be shuffled

## Train Baseline Model (VGG19)

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
vgg = tf.keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(244, 244, 3)
)


In [ ]:
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
x = tf.keras.layers.Flatten()(vgg.output)  # One flatten layer
prediction = tf.keras.layers.Dense(2, activation='softmax')(x) # one softmax layer
model = tf.keras.Model(inputs=vgg.input, outputs=prediction)   # Final Model
model.summary()

In [ ]:
# This needs to be checked
# We need to know which data to be added exactly. We want the Image Data Generator function but without any agumentation (for now)
# did not have time to check exactly how. 
# Try to compare with the paper stuff. Hopefully we will be able to find something.
from tensorflow.keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint(filepath='breast_histopathology_baseline_AUG.hdf5', verbose=1, save_best_only=True)

class_weights={
    0: 1.0, # negative
    1: 1.0, # postive 
    # make model more sensitive to positive class if necessary
}

model.compile(loss='categorical_crossentropy', optimizer='RMSProp', metrics=['accuracy']) 
history = model.fit(train_batches_bench, 
                                            steps_per_epoch=train_steps, 
                                            class_weight=class_weights,
                                            validation_data=valid_batches_bench,
                                            validation_steps=val_steps,
                                            callbacks=[model_checkpoint], 
                                            epochs=30, 
                                            verbose=1)

In [ ]:
import matplotlib.pyplot as plt
training_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epoch_count = range(1, 31)

plt.figure()
plt.plot(epoch_count, training_acc, 'r--')
plt.plot(epoch_count, val_acc, 'b-')
plt.legend(['Training Acc', 'Validation Acc'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.savefig("../vgg19-baseline-model-AUG.png")
plt.show()


In [ ]:
# Load Weights of the Baseline Model
model.load_weights('breast_histopathology_baseline_AUG.hdf5')

In [ ]:
# Test Predictions
preds = model.predict_generator(test_batches_bench, steps=len(df_test), verbose=1)

In [ ]:
# Accuracy using the baseline model
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
# Take the highest probability scores to get index of predictions for test images
y_pred = np.argmax(preds, axis=1)

# Get labels of test images
y_true = test_batches_bench.classes

# Evaluate
print(f'Accuracy: {accuracy_score(y_true, y_pred):1.3f}%')

print(f'Balanced Accuracy: {balanced_accuracy_score(y_true, y_pred):1.3f}%')
# Balanced accuracy is calculated as the average of the proportion correct of each class individually

f = open("../testOutput_AUG.txt", "w")
f.write(f'Accuracy: {accuracy_score(y_true, y_pred):1.3f}%\n')
f.write(f'Balanced Accuracy: {balanced_accuracy_score(y_true, y_pred):1.3f}%\n')
f.close()

In [ ]:
# Confusion matrix for the baseline model
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_true, y_pred)
plt.imshow(conf_matrix, interpolation='nearest', cmap='viridis')
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(['negative', 'positive']))
plt.xticks(tick_marks, ['negative', 'positive'])
plt.yticks(tick_marks, ['negative', 'positive'])
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.savefig("../vgg19-baseline-model-confusion-matrix-AUG.png")

In [ ]:
confusionmatrix = confusion_matrix(y_true, y_pred)

f = open("../testOutput_AUG.txt", "a")
f.write(f'Confusion matrix: {confusionmatrix}\n')
f.close()

confusionmatrix

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_true=y_true, y_pred=y_pred, target_names=['negative', 'positive'])

f = open("../testOutput_AUG.txt", "a")
f.write(f'REPORT:\n {report}\n')
f.close()

print(report)

In [ ]:
x, y = valid_batches_bench.next()

for i in range(1,10):
  
  image = x[i]
  label = y[i]
  f = open("../testOutput_AUG.txt", "a")
  if np.argmax(label, axis=0) == 0:
      print('IDC negative')
      f.write(f'IDC negative\n')
  if np.argmax(label, axis=0) == 1:
    print('IDC positive')
    f.write(f'IDC positive\n')

#   print(image.shape)
#   prob = model.predict(image)
#   for a in prob:
#     for b in a:
#       print(f'Predicted Negative Probability: {(b*100):1.2f}%')
#       print(f'Predicted Positive Probability: {(1-b)*100:1.2f}%')
#       f.write(f'Predicted Negative Probability: {(b*100):1.2f}%\n')
#       f.write(f'Predicted Positive Probability: {(1-b)*100:1.2f}%\n')
#       break

  plt.imshow(image)
#   f.write(image)
  plt.show()
  f.close()